<a href="https://colab.research.google.com/github/Xu-Kai-CUHKSZ/Word2Vec/blob/main/Word2Vec_NNLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


# 神经网络语言模型（NNLM）
- 本文参考[语言模型（二）—— 神经网络语言模型（NNLM）](https://blog.csdn.net/rongsenmeng2835/article/details/108571335) <p>

**语言模型**：语言模型的作用就是检测一句话是不是正常人说出来的。语言模型形像化的解释是：给定一句话，看它是自然语言的概率P(w1,w2,w3,..,wt)是多少。<p>
**词向量**：在神经网络语言模型中，词向量作为一个内部参数，跟神经网络中的其他内部参数一样都是先有一个随机初始化值，正向传播后计算损失函数再反向传播更新这些参数。这也就要求神经网络语言模型是有监督的学习，词向量是学习得到的副产物，也是模型内化的一部分。

![NNLM网络结构图](https://drive.google.com/uc?id=1VWXaZ9PQJsjD1g4Djhd6UDSj6Vrl9LKX)

### 模型参数
神经网络真正的输入： $ x = (C(w_{t-n+1}),C(w_{t-n+2}),...,C(w_{t-1})) $ <p>
输出：$ y = y_{1} + y_{2} = b + Wx + Utanh(d+Hx)$
- 这里 $y$的维度应该等于 $V$,即词汇表的大小。这样再将 $y$经过一个softmax函数做概率归一化，便能得到一个维度为$V$的概率向量，这就是我们的输出了。（找到最大的概率所在位置的索引，结合词表我们就能得到我们的预测值了）

模型训练的目标是最大化以下似然函数：$L = \frac{1}{T} \sum_{t} log f(w_i, w_{i-1}, ..., w_{i-n+1}; \theta) + R(\theta)$ , 其中

$$\theta = (b, d, W, U, H, C).$$

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
"""
1.Basic Embedding Model
    1-1. NNLM(Neural Network Language Model)
"""

dtype = torch.FloatTensor
sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()  # 制作词汇表， " ".join(sentences)：将句子列表中的所有句子合并为一个单一的字符串
print(word_list)
word_list = list(set(word_list))  # 去除词汇表中的重复元素
print("去重后的word_list:", word_list)
print(' ')
word_dict = {w: i for i, w in enumerate(word_list)}  # 将每个单词对应于相应的索引
number_dict = {i: w for i, w in enumerate(word_list)}  # 将每个索引对应于相应的单词
n_class = len(word_dict)  # 单词的总数
print(f'n_class:{n_class}')
print(' ')
print(f'word_dict:{word_dict}')
print(' ')
print(f'numberr_dict:{number_dict}')

# NNLM parameters
n_step = 2   # 根据前两个单词预测第3个单词
n_hidden = 2  # 隐藏层神经元的个数
m = 2  # 词向量的维度

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']
去重后的word_list: ['dog', 'like', 'love', 'coffee', 'milk', 'i', 'hate']
 
n_class:7
 
word_dict:{'dog': 0, 'like': 1, 'love': 2, 'coffee': 3, 'milk': 4, 'i': 5, 'hate': 6}
 
numberr_dict:{0: 'dog', 1: 'like', 2: 'love', 3: 'coffee', 4: 'milk', 5: 'i', 6: 'hate'}


In [ ]:
# 由于pytorch中输入的数据是以batch小批量进行输入的，下面的函数就是将原始数据以一个batch为基本单位喂给模型
def make_batch(sentences):
    """
    input:
    sentences = ["i like dog", "i love coffee", "i hate milk"]
    output:
    input_batch = [[1, 4], [1, 3], [1, 5]]
    target_batch = [0, 6, 2]
    """
    input_batch = []
    target_batch = []
    for sentence in sentences:
        word = sentence.split()
        input = [word_dict[w] for w in word[:-1]]   # 获取前n-1个单词
        target = word_dict[word[-1]]           # 获取第n个单词
        input_batch.append(input)
        target_batch.append(target)
    return input_batch, target_batch

In [ ]:
input_batch, target_batch = make_batch(sentences)
print(f'input_batch: {input_batch}')
print(f'target_batch: {target_batch}')
input_batch = torch.LongTensor(input_batch)
print(input_batch.size())
print(input_batch)

input_batch: [[5, 1], [5, 2], [5, 6]]
target_batch: [0, 3, 4]
torch.Size([3, 2])
tensor([[5, 1],
        [5, 2],
        [5, 6]])


输出：$ y = y_{1} + y_{2} = b + Wx + Utanh(d+Hx)$

In [ ]:
# Model
class NNLM(nn.Module):    # 定义一个名为 NNLM 的类，它继承自 nn.Module。这是PyTorch中构建神经网络的基类
    def __init__(self):
        # super() 是一个内置函数，用于调用父类（这里是 nn.Module）的方法
        # super(NNLM, self) 返回 NNLM 类的父类（nn.Module）的一个代理对象，通过这个代理，可以调用父类的方法。
        # .__init__() 调用父类的初始化方法，确保父类的所有初始化步骤都被执行，特别是那些在 nn.Module 中定义的。
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, embedding_dim=m)  # 创建一个词嵌入层，用于将单词的索引映射到稠密的词向量, C这里表示词汇表矩阵
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))   # 隐藏层权重  H 为 4 x 2的矩阵
        self.W = nn.Parameter(torch.randn(n_step * m, n_class).type(dtype))   # 定义矩阵W 4 x 7
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))          # 定义d:2
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))     # U: 2x7
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))          # b:7

    def forward(self, x):
        # 输入x是一个包含单词索引的张量，经过词嵌入层后，x被转换为一个包含词向量的张量，形状为(batch_size=3, n_step = 2, m = 2)
        # 其中 n_step是上下文窗口的大小，m是词向量的维度。
        x = self.C(x)
        x = x.view(-1, n_step * m) # x: [batch_size, n_step*n_class]

        # torch.mm(x, self.H)：进行矩阵乘法，将输入 x（形状为 (batch_size, n_step * m)）与隐藏层权重 self.H（形状为 (n_step * m, n_hidden)）相乘
        # 结果是tanh一个形状为 (batch_size, n_hidden) 的张量。
        tanh = torch.tanh(self.d + torch.mm(x, self.H))


        output = self.b + torch.mm(x, self.W) + torch.mm(tanh, self.U)
        # output: [batch_size, n_class]
        return output


In [ ]:
model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 制作输入
input_batch, target_batch = make_batch(sentences)
# 将输入和目标转换为 LongTensor
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

# 开始训练
for epoch in range(5000):
    optimizer.zero_grad()    # 清零梯度，清除上一个周期计算的梯度。由于 PyTorch 默认会累积梯度，所以每个周期开始时需要将梯度清零。
    output = model(input_batch)
    # output : [batch_size=3, n_class=7], target_batch : [batch_size=3] (LongTensor, not one-hot)

    loss = criterion(output, target_batch)   # 计算损失
    if (epoch + 1) % 1000 == 0:    # 每 1000 个周期输出一次当前的周期数和损失值，以便监控训练过程
        print("Epoch:{}".format(epoch + 1), "Loss:{:.3f}".format(loss))
    loss.backward()    # 执行反向传播，计算损失相对于模型参数的梯度
    optimizer.step()   #更新模型的参数，根据计算得到的梯度调整参数，以减少损失

# 预测
# data.max(1, keepdim=True)：在每一行（每个样本）中找到最大值及其索引，返回一个元组，其中第二个元素是最大值的索引（即预测的类别）。
# [1] 选择索引，得到预测类别的张量 predict，其形状为 [batch_size, 1]。
predict = model(input_batch).data.max(1, keepdim=True)[1]  # [batch_size, n_class]
print("predict: \n", predict)
# 测试
print([sentence.split()[:2] for sentence in sentences], "---->",
      [number_dict[n.item()] for n in predict.squeeze()])


Epoch:1000 Loss:0.109
Epoch:2000 Loss:0.020
Epoch:3000 Loss:0.007
Epoch:4000 Loss:0.003
Epoch:5000 Loss:0.001
predict: 
 tensor([[0],
        [3],
        [4]])
[['i', 'like'], ['i', 'love'], ['i', 'hate']] ----> ['dog', 'coffee', 'milk']


In [25]:
# 获取训练后的词嵌入
word_embeddings = model.C.weight.data
print(word_embeddings)

tensor([[ 2.4772, -0.0403],
        [-0.1820, -0.5141],
        [-0.4413,  2.4713],
        [-0.5664,  0.4791],
        [-1.2249, -0.6533],
        [ 0.5640,  1.1866],
        [ 1.0607,  0.2699]])
